In [1]:
import rg
from rg.interaction import interaction as J
from rg.interaction import interaction_system
from rg.diagrams import diagram

In [2]:
j = J([ [1,1],  [1,0]] )

In [3]:
j.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [4]:
interactions = [
 J([[1,1],[0,0]]),
 J([[0,0],[1,1]]),
 J([[1,2],[0,0]]),
 J([[1,0],[0,1]]),
 J([[1,2],[0,0]]),
 J([[1,1],[0,1]]),
 ]

IS = interaction_system(interactions)
IS.display()

⎡                                                             2               
⎣\phi₀⋅\tilde{\phi}₀, \phi₁⋅\tilde{\phi}₁, \phi₀⋅\tilde{\phi}₀ , \phi₀⋅\tilde{

                                         ⎤
\phi}₁, \phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁⎦

In [5]:
I = J([ [1,1],  [0,0]] )
Lambda = rg.T**(-1) # L**(-1*dim)
new_couplings = {I:Lambda,
                 J([ [0,0],  [1,1]] ): Lambda,
                 J([ [1,2],  [0,0]] ):1,
                 J([ [1,1],  [0,1]] ):1,
                }
IS.reduce(new_couplings)
IS.__display_couplings__()

⎡⎡                     1⎤  ⎡                     1⎤  ⎡                   2   ⎤
⎢⎢\phi₀⋅\tilde{\phi}₀, ─⎥, ⎢\phi₁⋅\tilde{\phi}₁, ─⎥, ⎣\phi₀⋅\tilde{\phi}₀ , 1⎦
⎢⎣                     T⎦  ⎣                     T⎦                           
⎣                                                                             

  ⎡                     1⎤                                          ⎡         
, ⎢\phi₀⋅\tilde{\phi}₁, ─⎥, [\phi₀⋅\tilde{\phi}₀⋅\tilde{\phi}₁, 1], ⎣\tilde{\p
  ⎣                     T⎦                                                    
                                                                              

       -d⎤  ⎡       1 ⎤  ⎡             2   -d  ⎤  ⎡                -d⎤  ⎡     
hi}₀, L  ⎦, ⎢\phi₀, ──⎥, ⎣\tilde{\phi}₀ , L  ⋅T⎦, ⎣\tilde{\phi}₁, L  ⎦, ⎢\phi₀
            ⎢        2⎥                                                 ⎣     
            ⎣       T ⎦                                                       

                1⎤  ⎡                            